In [15]:
pip install newsapi-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import requests
from newsapi import NewsApiClient

# Replace 'your_api_key_here' with your actual NewsAPI key
api_key = '5adaa15dd0fe42b7bc567ddff65d3fe8'
newsapi = NewsApiClient(api_key=api_key)

EUR_forex_keywords = ['EUR', 'Euro', 'euro','€']
JPY_forex_keywords = ['JPY', 'Yen', 'yen', '¥']
GBP_forex_keywords = ['GBP', 'Pound', 'pound', '£']
AUD_forex_keywords = ['AUD', 'Australian Dollar', 'australian dollar', 'A$', 'AU$']
CAD_forex_keywords = ['CAD', 'Canadian Dollar', 'canadian dollar', 'Can$', 'C$']

exchanges = [EUR_forex_keywords, JPY_forex_keywords, GBP_forex_keywords, AUD_forex_keywords, CAD_forex_keywords]

targetEXCH = EUR_forex_keywords

forex_related_news = {}

for exchange in exchanges:
  forex_keywords = exchange

  for keyword in forex_keywords:
      forex_related_news[keyword] = []

  # Fetch top headlines
  top_headlines = newsapi.get_everything(q = 'forex', language='en', sort_by='relevancy', page_size=100)

  # Filter and categorize forex-related articles
  for article in top_headlines['articles']:
      title = article['title'] if article['title'] else ""
      description = article['description'] if article['description'] else ""
      content = article['content'] if article['content'] else ""
      date = article['publishedAt'] if article['publishedAt'] else ""

      for keyword in forex_keywords:
          if (keyword in title) or (keyword in description) or (keyword in content):
              forex_related_news[keyword].append(article)
              break

  # Print categorized news articles
  for keyword, articles in forex_related_news.items():
      print(f"Forex articles related to {keyword} to USD:")
      for article in articles:
          print(f"Title: {article['title']}")
          print(f"Source: {article['source']['name']}")
          print(f"URL: {article['url']}")
          print(f"Date: {article['publishedAt']}")
          print()
      print("-" * 80)

      print(forex_related_news)

  print(f"\n\n")


Forex articles related to EUR to USD:
Title: EUR/USD Forecast – Euro Rallies Towards Resistance
Source: Yahoo Entertainment
URL: https://finance.yahoo.com/news/eur-usd-forecast-euro-rallies-124216860.html
Date: 2023-03-28T12:42:16Z

--------------------------------------------------------------------------------
{'EUR': [{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Christopher Lewis', 'title': 'EUR/USD Forecast – Euro Rallies Towards Resistance', 'description': 'The Euro has rallied a bit against the US dollar during the trading session on Tuesday, as we continue to see a lot of choppiness in the Forex world.', 'url': 'https://finance.yahoo.com/news/eur-usd-forecast-euro-rallies-124216860.html', 'urlToImage': 'https://media.zenfs.com/en/fx_empire_176/fb6af035cfd6639d47aa7fd18cb9c611', 'publishedAt': '2023-03-28T12:42:16Z', 'content': 'EUR/USD Forecast Video for 29.03.23\r\nEuro vs US Dollar Technical Analysis\r\nThe Euro has rallied a bit against the US dollar dur

In [18]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove special characters and numbers
    text = re.sub(r'\W+|\d+', ' ', text)
    
    # Remove underscores
    text = re.sub(r'_', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

def lemmatize(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

def preprocess_text(text):
    text = clean_text(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens)
    return tokens



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# Initialize a dictionary to store preprocessed_texts for each exchange
preprocessed_texts_by_exchange = {}

for exchange in exchanges:
    # Extract the texts from the news articles
    news_texts = [article['title'] + ' ' + article['description'] for keyword in exchange for article in forex_related_news[keyword]]

    # Preprocess the texts
    preprocessed_texts = [preprocess_text(text) for text in news_texts]

    # Store the preprocessed_texts in the preprocessed_texts_by_exchange dictionary
    exchange_name = exchange[0]  # Use the first keyword as the exchange name
    preprocessed_texts_by_exchange[exchange_name] = preprocessed_texts

    # Print an example
    print(f'Exchange: {exchange_name}')
    print(f'Original texts:')
    for text in news_texts:
        print(text)
        print()

    print(f'Preprocessed texts:')
    for preprocessed_text in preprocessed_texts:
        print(preprocessed_text)
        print()

    print('-' * 80)


Exchange: EUR
Original texts:
EUR/USD Forecast – Euro Rallies Towards Resistance The Euro has rallied a bit against the US dollar during the trading session on Tuesday, as we continue to see a lot of choppiness in the Forex world.

Euro to the fore as hawkish hike looms The euro rose above $1.10 overnight and was still going at $1.1062 early in the Asia session. Trade was thinned by holidays in Australia and New Zealand and ahead of central bank meetings in Japan on Friday and the U.S. and Europe next week.

India's forex reserves up $6.31 billion to $584.76 billion, at 9-month high Foreign Currency Assets increased by USD 4.740 billion to USD 514.431 billion. Expressed in dollar terms, the foreign currency assets include the effect of appreciation or depreciation of non-US units like the euro, pound and yen held in the foreign exchange …

India's forex reserves rise by USD 6.3 billion to USD 584.76 billion Foreign Currency Assets increased by USD 4.740 billion to USD 514.431 billion. 

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tokens_to_text(tokens):
    return ' '.join(tokens)

# Initialize a dictionary to store the tfidf_matrix for each exchange
tfidf_matrices_by_exchange = {}

for exchange_name, preprocessed_texts in preprocessed_texts_by_exchange.items():
    # Join tokens into a single string for each preprocessed text
    preprocessed_texts_joined = [tokens_to_text(tokens) for tokens in preprocessed_texts]

    if len(preprocessed_texts_joined) > 0:
        vectorizer = TfidfVectorizer()

        # Fit the vectorizer on the preprocessed texts and transform the texts to feature vectors
        tfidf_matrix = vectorizer.fit_transform(preprocessed_texts_joined)

        # Store the tfidf_matrix in the tfidf_matrices_by_exchange dictionary
        tfidf_matrices_by_exchange[exchange_name] = tfidf_matrix

        # Print the shape of the feature matrix
        print(f'{exchange_name} TF-IDF feature matrix shape: {tfidf_matrix.shape}')
    else:
        print(f'No articles found for {exchange_name}')


EUR TF-IDF feature matrix shape: (5, 87)
JPY TF-IDF feature matrix shape: (2, 27)
GBP TF-IDF feature matrix shape: (2, 27)
No articles found for AUD
No articles found for CAD


In [24]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

def get_sentiment(text):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score

# Initialize a dictionary to store the sentiment scores for each exchange
sentiment_scores_by_exchange = {}

for exchange_name, preprocessed_texts in preprocessed_texts_by_exchange.items():
    # Join tokens into a single string for each preprocessed text
    preprocessed_texts_joined = [tokens_to_text(tokens) for tokens in preprocessed_texts]

    if len(preprocessed_texts_joined) > 0:
        sentiment_scores = [get_sentiment(text) for text in preprocessed_texts_joined]

        # Store the sentiment scores in the sentiment_scores_by_exchange dictionary
        sentiment_scores_by_exchange[exchange_name] = sentiment_scores

        # Print the sentiment scores
        print(f'Sentiment scores for {exchange_name}:')
        for i in range(len(sentiment_scores)):
            print(f'Article {i+1} sentiment: {sentiment_scores[i]}')
        print('-' * 80)
    else:
        print(f'No articles found for {exchange_name}')


Sentiment scores for EUR:
Article 1 sentiment: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Article 2 sentiment: {'neg': 0.062, 'neu': 0.85, 'pos': 0.088, 'compound': 0.2023}
Article 3 sentiment: {'neg': 0.0, 'neu': 0.692, 'pos': 0.308, 'compound': 0.8979}
Article 4 sentiment: {'neg': 0.0, 'neu': 0.699, 'pos': 0.301, 'compound': 0.8979}
Article 5 sentiment: {'neg': 0.0, 'neu': 0.895, 'pos': 0.105, 'compound': 0.5267}
--------------------------------------------------------------------------------
Sentiment scores for JPY:
Article 1 sentiment: {'neg': 0.0, 'neu': 0.692, 'pos': 0.308, 'compound': 0.8979}
Article 2 sentiment: {'neg': 0.0, 'neu': 0.699, 'pos': 0.301, 'compound': 0.8979}
--------------------------------------------------------------------------------
Sentiment scores for GBP:
Article 1 sentiment: {'neg': 0.0, 'neu': 0.692, 'pos': 0.308, 'compound': 0.8979}
Article 2 sentiment: {'neg': 0.0, 'neu': 0.699, 'pos': 0.301, 'compound': 0.8979}
----------------------------

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [25]:
# Median compound computer
for exchange_name, sentiment_scores in sentiment_scores_by_exchange.items():
    median_compound = 0

    for score in sentiment_scores:
        median_compound += score['compound']

    # Calculate the median compound sentiment for the current exchange
    median_compound /= len(sentiment_scores)

    print(f"Median Compound for {exchange_name}: {median_compound}")

    if median_compound > 0:
        print("Positive")
    elif median_compound < 0:
        print("Negative")
    else:
        print("Neutral")

    print('-' * 80)


Median Compound for EUR: 0.50496
Positive
--------------------------------------------------------------------------------
Median Compound for JPY: 0.8979
Positive
--------------------------------------------------------------------------------
Median Compound for GBP: 0.8979
Positive
--------------------------------------------------------------------------------
